<a href="https://colab.research.google.com/github/donghwi2022/ds-section1-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다음 분기에 어떤 게임을 설계해야 할까

# **Part 1. 필요 라이브러리 및 데이터 로드**
- 필요 라이브러리 import 및 한글 깨짐문제 해결
- csv 파일 load

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#한글 깨짐문제 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic')

In [ ]:
# url로 필요 csv 파일 불러오기
url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv"
df = pd.read_csv(url)
df.head()

# **Part 2. 데이터 전처리 진행**

In [ ]:
df.info()

In [ ]:
# 첫번째 column은 index를 나타내므로 해당 열 제거
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

In [ ]:
# 결측치의 존재 확인
df.isnull().sum()

In [ ]:
# 데이터 양이 충분하므로, 결측치가 존재하는 데이터 삭제
df.dropna(inplace = True)
df = df.reset_index()
df.info()